# 6. 의사결정나무

## 6.1 핵심 개념

**의사결정나무(decision tree)는 의사결정 규칙을 나무구조로 도표화 하여 관심 대상이 되는 집단을 몇 개의 소집단으로 분류 하거나 특정값을 예측하는데 활용되는 분석기법**이다.


![의사결정나무](./extrafiles/decisiontree.png)

**[구성요소]**

![트리](./extrafiles/tree.png)

|구성요소	|설명|
|:--|:--|
|뿌리 마디(Root Node)	|나무가 시작되는 마디로 전체 자료를 포함|
|중간 마디(InternalNode)|	부모와 자식 마디를 모두 가진 마디|
|끝 마디(Terminal Node)	|자식 노드가 없는 마디|
|부모 마디(Parent Node)	|주어진 노드의 상위 마디|
|자식 마디(Child Node)	|주어진 노드의 하위 마디|
|가지(Branch)	|하나의 마디로부터 끝 마디까지 연결된 마디들|
|깊이(Depth)	|가지를 이루는 마디의 개수|

**[주요특징]**  

**의사결정트리는 직관적으로 결과를 도식화 하여 볼 수 있다는 장점**이 있습니다. 그러나 <u>분류 단계가 증가할 수록 트리는 복잡해지며 데이터에 따라 안정적인 결과도출이 힘들어 짐에 따라 머신러닝 알고리즘으로 많이 활용되지는 않습니다.</u> 하지만 특성치가 많지 않고 최종 알고리즘을 도출하기 전 탐색적으로 주요 변수가 어떤것인지를 도출 하는 차원에서는 활용 하기 좋은 알고리즘입니다.

## 6.2 scikit-learn

**의사결정나무는 sklearn.tree 패키지**에 속해 있습니다. **DecisionTreeClassifier 는 분류 알고리즘**이며, **DecisionTreeRegressor 는 회귀분석 알고리즘**입니다.


|sklearn.tree|Decision Trees|
|:--|:--|
|sklearn.tree.DecisionTreeClassifier() |A decision tree classifier. |
|sklearn.tree.DecisionTreeRegressor() |A decision tree regressor. |
|sklearn.tree.ExtraTreeClassifier() |An extremely randomized tree classifier. |
|sklearn.tree.ExtraTreeRegressor() |An extremely randomized tree regressor. |


의사결정나무의 하이퍼 파라미터중 분류나 회귀 결과에 영향을 미치는 것은 **max_depth, max_leaf_node, min_sample_leaf** 입니다.

|Hyper Parameter||
|:--|:--|
|max_depth|최대 가지치기의 수|
|max_leaf_node|리프 노드의 최대 개수|
|max_sample_leaf|리프 노트가 되기 위한 최소 샘플 수|


## 6.3 분석 코드

### Part1. 분류(Classification)

In [11]:
# 경고레벨조정
import warnings
warnings.filterwarnings("ignore")

# 데이터 로드
import pandas as pd
data = pd.read_csv("./extrafiles/breast-cancer-wisconsin.csv", encoding='utf-8')

# 컬럼정보 확인
print(data.columns)

# 독립변수/ 종속변수 분리
X = data[['Clump_Thickness', 'Cell_Size', 'Cell_Shape',
       'Marginal_Adhesion', 'Single_Epithelial_Cell_Size', 'Bare_Nuclei',
       'Bland_Chromatin', 'Normal_Nucleoli', 'Mitoses']]
y = data[['Class']]

# train-test data 분리
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

# stratify 효과 - 범주형 변수를 유사한 비율로 train / test 데이터로 분리시켜 준다.
print(y_train.mean())
print(y_test.mean())

# 표준화 작업 - MinMaxScaler
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(X_train)

X_scaled_train = scaler.transform(X_train)
X_scaled_test = scaler.transform(X_test)

Index(['code', 'Clump_Thickness', 'Cell_Size', 'Cell_Shape',
       'Marginal_Adhesion', 'Single_Epithelial_Cell_Size', 'Bare_Nuclei',
       'Bland_Chromatin', 'Normal_Nucleoli', 'Mitoses', 'Class'],
      dtype='object')
Class    0.349609
dtype: float64
Class    0.350877
dtype: float64


In [12]:
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier()
model.fit(X_scaled_train, y_train)
pred_train = model.predict(X_scaled_train)
model.score(X_scaled_train, y_train)

1.0

In [13]:
# 훈련데이터의 혼동행렬 작성
from sklearn.metrics import confusion_matrix
confusion_train = confusion_matrix(y_train, pred_train)
print("훈련데이터 오차행렬 : \n", confusion_train)

훈련데이터 오차행렬 : 
 [[333   0]
 [  0 179]]


In [14]:
# 훈련데이터의 분류 레포트 작성
from sklearn.metrics import classification_report
cfreport_train = classification_report(y_train, pred_train)
print("분류예측 레포트 : \n", cfreport_train)

분류예측 레포트 : 
               precision    recall  f1-score   support

           0       1.00      1.00      1.00       333
           1       1.00      1.00      1.00       179

    accuracy                           1.00       512
   macro avg       1.00      1.00      1.00       512
weighted avg       1.00      1.00      1.00       512



훈련데이터의 정확도는 100%에 해당한다.

In [15]:
# 테스트데이터 예측결과 생성
pred_test = model.predict(X_scaled_test)
model.score(X_scaled_test, y_test)

0.9532163742690059

In [16]:
# 테스트데이터의 혼동행렬 작성
confusion_test = confusion_matrix(y_test, pred_test)
print("테스트데이터 오차행렬 : \n", confusion_test)

테스트데이터 오차행렬 : 
 [[105   6]
 [  2  58]]


In [17]:
# 테스트데이터의 분류 레포트 작성
from sklearn.metrics import classification_report
cfreport_test = classification_report(y_test, pred_test)
print("분류예측 레포트 : \n", cfreport_test)

분류예측 레포트 : 
               precision    recall  f1-score   support

           0       0.98      0.95      0.96       111
           1       0.91      0.97      0.94        60

    accuracy                           0.95       171
   macro avg       0.94      0.96      0.95       171
weighted avg       0.95      0.95      0.95       171



테스트데이터의 정확도는 95.3%에 해당된다.

In [22]:
# 하이퍼 파라미터 튜닝 - Grid Search
from sklearn.model_selection import GridSearchCV
param_grid = {'max_depth'        :range(2, 20, 2), 
              'min_samples_leaf' :range(1, 50, 2)}
             
grid_search = GridSearchCV(DecisionTreeClassifier(), param_grid, cv=5)
grid_search.fit(X_scaled_train, y_train)

GridSearchCV(cv=5, estimator=DecisionTreeClassifier(),
             param_grid={'max_depth': range(2, 20, 2),
                         'min_samples_leaf': range(1, 50, 2)})

In [23]:
# 파라미터 튜닝 결과 확인
print("Best Parameter : {}".format(grid_search.best_params_))
print("Best Score : {:.4f}".format(grid_search.best_score_))
print("Test set Score : {:.4f}".format(grid_search.score(X_scaled_test, y_test)))

Best Parameter : {'max_depth': 18, 'min_samples_leaf': 1}
Best Score : 0.9667
Test set Score : 0.9474


In [26]:
# 하이퍼 파라미터 튜닝2 - Randomized Search
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

param_distribs = {'max_depth'        :randint(low=1, high=20), 
                  'min_samples_leaf' :randint(low=1, high=50)}

random_search = RandomizedSearchCV(DecisionTreeClassifier(), param_distributions=param_distribs, n_iter=20, cv=5)
random_search.fit(X_scaled_train, y_train)

RandomizedSearchCV(cv=5, estimator=DecisionTreeClassifier(), n_iter=20,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_frozen object at 0x0000017172F7B790>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_frozen object at 0x0000017176915E20>})

In [27]:
# 파라미터 튜닝 결과값 확인
print("Best Parameter : {}".format(random_search.best_params_))
print("Best Score : {:.4f}".format(random_search.best_score_))
print("Test set Score : {:.4f}".format(random_search.score(X_scaled_test, y_test)))

Best Parameter : {'max_depth': 15, 'min_samples_leaf': 6}
Best Score : 0.9531
Test set Score : 0.9591


[종합정리]

**의사결정나무는 학습데이터에 매우 과정합되는 경향**이 있습니다. 앞선 100% 예측율에 비해 테스트 데이터에서는 95% 예측율로 낮아지는것이 그 예입니다. 또한 **하이퍼 파라미터 설정에 대한 기준 역시 논리적으로 추론하기가 어렵**습니다. 이러한 한계에도 불구하고 의사결정 트리는 <u>기본적인 분석 방향을 탐색하기 위한 초기 분석 모델로서는 유용</u>합니다. 실제 분석에서는 의사결정나무와 앙상블된 랜덤 포레스트가 주로 사용됩니다.

### Part2. 회귀(Regression)

In [28]:
# 데이터 로드
data2 = pd.read_csv('./extrafiles/house_price.csv', encoding='utf-8')

print(data2.columns)

X = data2[data2.columns[1:5]]
y = data2[['house_value']]

print(X.columns)

# train-test data 분리
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

# stratify 효과 - 범주형 변수를 유사한 비율로 train / test 데이터로 분리시켜 준다.
print(y_train.mean())
print(y_test.mean())

# 표준화 작업 - MinMaxScaler
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(X_train)

X_scaled_train = scaler.transform(X_train)
X_scaled_test = scaler.transform(X_test)

Index(['housing_age', 'income', 'bedrooms', 'households', 'rooms',
       'house_value'],
      dtype='object')
Index(['income', 'bedrooms', 'households', 'rooms'], dtype='object')
house_value    189260.967812
dtype: float64
house_value    188391.001357
dtype: float64


In [34]:
# 의사결정트리 모델 적용
from sklearn.tree import DecisionTreeRegressor
model = DecisionTreeRegressor()
model.fit(X_scaled_train, y_train)
pred_train = model.predict(X_scaled_train)
model.score(X_scaled_train, y_train)

1.0

In [35]:
# 테스트 데이터 모델 적용
pred_test = model.predict(X_scaled_test)
model.score(X_scaled_test, y_test)

0.21465026278419952

In [36]:
# 회귀분석의 지표 R Square n RMSE
# RMSE (Root Mean Squared Error)
from sklearn.metrics import mean_squared_error
import numpy as np
MSE_train = mean_squared_error(y_train, pred_train)
MSE_test = mean_squared_error(y_test, pred_test)
print("훈  련데이터 RMSE:", np.sqrt(MSE_train))
print("테스트데이터 RMSE:", np.sqrt(MSE_test))

훈  련데이터 RMSE: 0.0
테스트데이터 RMSE: 84721.57586552504


훈련 데이터의 오차율이 0인것에 비래 테스트 데이터에서는 그 차이가 많이 납니다. 이것은 **훈련데이터에 과적합(overfitting)된 예**라고 볼 수 있습니다.

In [39]:
# 하이퍼 파라미터 튜닝 - Grid Search
from sklearn.model_selection import GridSearchCV
param_grid = {'max_depth'        :range(2, 20, 2), 
              'min_samples_leaf' :range(1, 50, 2)}
grid_search = GridSearchCV(DecisionTreeRegressor(), param_grid, cv=5)
grid_search.fit(X_scaled_train, y_train)

GridSearchCV(cv=5, estimator=DecisionTreeRegressor(),
             param_grid={'max_depth': range(2, 20, 2),
                         'min_samples_leaf': range(1, 50, 2)})

In [40]:
# 파라미터 튜닝 결과 확인
print("Best Parameter : {}".format(grid_search.best_params_))
print("Best Score : {:.4f}".format(grid_search.best_score_))
print("Test set Score : {:.4f}".format(grid_search.score(X_scaled_test, y_test)))

Best Parameter : {'max_depth': 8, 'min_samples_leaf': 49}
Best Score : 0.5592
Test set Score : 0.5770


In [41]:
# 하이퍼 파라미터 튜닝2 - Randomized Search
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

param_distribs = {'max_depth'        :randint(low=1, high=20), 
                  'min_samples_leaf' :randint(low=1, high=50)}

random_search = RandomizedSearchCV(DecisionTreeRegressor(), param_distributions=param_distribs, n_iter=20, cv=5)
random_search.fit(X_scaled_train, y_train)

RandomizedSearchCV(cv=5, estimator=DecisionTreeRegressor(), n_iter=20,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_frozen object at 0x0000017176B2D880>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_frozen object at 0x0000017176B2D2E0>})

In [42]:
# 파라미터 튜닝 결과값 확인
print("Best Parameter : {}".format(random_search.best_params_))
print("Best Score : {:.4f}".format(random_search.best_score_))
print("Test set Score : {:.4f}".format(random_search.score(X_scaled_test, y_test)))

Best Parameter : {'max_depth': 10, 'min_samples_leaf': 47}
Best Score : 0.5576
Test set Score : 0.5768


**[종합정리]**

회귀분석 문제에서도 과적합 문제가 발생되었습니다. 하지만 **적절한 하이퍼파라미터만 찾는다면 안정적인 예측율을 찾을 수 있고 일반화역시 가능**합니다. <u>다만 항시 하이퍼 파라미터를 기본 값이 아닌 적절값을 설정 해주어야만 합니다.</u>